<a href="https://colab.research.google.com/github/jacksontenorio8/FakeNews/blob/main/FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
print(stopwords.words('portuguese'))

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [10]:
df = pd.read_csv('/content/drive/MyDrive/fakenews/true.csv', encoding='utf-8')

# Nova secção

In [11]:
df.head()

,title,text,origin,url,label,publisher_name,publisher_site,date
0,Jovem Pan: o que é falso e o que é verdadeiro ...,As propostas de manter zerados impostos federa...,Jair Bolsonaro,https://lupa.uol.com.br/jornalismo/2022/09/06/...,0,Lupa - UOL,lupa.uol.com.br,NaN
1,Jovem Pan: o que é falso e o que é verdadeiro ...,Estatais dão lucro de mais de R$ 100 bilhões e...,Jair Bolsonaro,https://lupa.uol.com.br/jornalismo/2022/09/06/...,0,Lupa - UOL,lupa.uol.com.br,NaN
2,De cargos secretos a segurança: veja erros e a...,A água foi privatizada. A Cedae não acabou. Es...,Marcelo Freixo (PSB),https://lupa.uol.com.br/jornalismo/2022/09/01/...,0,Lupa - UOL,lupa.uol.com.br,NaN
3,De cargos secretos a segurança: veja erros e a...,São quase 3 milhões de pessoas passando fome [...,Marcelo Freixo (PSB),https://lupa.uol.com.br/jornalismo/2022/09/01/...,0,Lupa - UOL,lupa.uol.com.br,NaN
4,De cargos secretos a segurança: veja erros e a...,[Cláudio Castro] tem pouco mais de 20% de ótim...,Marcelo Freixo (PSB),https://lupa.uol.com.br/jornalismo/2022/09/01/...,0,Lupa - UOL,lupa.uol.com.br,NaN


In [12]:
df = pd.read_csv('/content/drive/MyDrive/fakenews/fakes.csv',encoding='utf-8')

In [13]:
df.isnull().sum()

title                0
text                 0
origin            4644
url                  0
label                0
publisher_name     363
publisher_site       0
date              3036
dtype: int64

In [14]:
df.head()

,title,text,origin,url,label,publisher_name,publisher_site,date
0,Vídeo com vaias a Nando Reis teve áudio manipu...,“Nando Reis se F… Se F…. no Rock in Rio”,pessoa,https://lupa.uol.com.br/jornalismo/2022/09/09/...,1,Lupa - UOL,lupa.uol.com.br,NaN
1,Documentos confirmam uso de dinheiro vivo na c...,"“Não foi dinheiro vivo. Moeda corrente, quer d...",Cristina Graeml,https://lupa.uol.com.br/jornalismo/2022/09/09/...,1,Lupa - UOL,lupa.uol.com.br,2022-09-06T00:00:00Z
2,G1 não publicou notícia de que rainha Elizabet...,"“Em seus últimos momentos de vida, rainha Eliz...",Imagem no WhatsApp,https://lupa.uol.com.br/jornalismo/2022/09/08/...,1,Lupa - UOL,lupa.uol.com.br,NaN
3,TSE não proibiu governo de tomar medidas para ...,“TSE MARCOU PARA PRÓXIMA SEMANA JULGAMENTO OND...,pessoa,https://lupa.uol.com.br/jornalismo/2022/09/08/...,1,Lupa - UOL,lupa.uol.com.br,NaN
4,É falso que órgão de saúde dos EUA passou a re...,"De repente, a Ivermectina aparece no site do N...",Post no WhatsApp,https://lupa.uol.com.br/jornalismo/2022/09/08/...,1,Lupa - UOL,lupa.uol.com.br,2022-09-08T00:00:00Z


In [15]:
df.isnull().sum()

title                0
text                 0
origin            4644
url                  0
label                0
publisher_name     363
publisher_site       0
date              3036
dtype: int64